In [1]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import numpy as np
from helper_functions.grape_functions import *
from main_grape.Grape import Grape
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
qubit_num = 3


#Defining time scales
total_time = 1.0
steps = 50


state_transfer = True
#Defining H0
qubit_state_num = 2
freq_ge = 3.9#GHz
g = 2*np.pi*0.5 #GHz

ens = np.array([ 2*np.pi*ii*(freq_ge) for ii in np.arange(qubit_state_num)])
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))
I_q = np.identity(qubit_state_num)
H_q = np.diag(ens)

g_op = nn_chain_kron(Q_x, I_q, qubit_num, qubit_state_num)

#g_op_test = np.kron(np.kron(np.kron(Q_x,Q_x),I_q),I_q) + np.kron(np.kron(np.kron(I_q,Q_x),Q_x),I_q) + np.kron(np.kron(np.kron(I_q,I_q),Q_x),Q_x)
#print np.allclose(g_op, g_op_test)

H0 = kron_all(H_q,qubit_num, I_q) + g * g_op

# Defining dressed info
is_dressed = True
w_c, v_c, dressed_id = get_dressed_info(H0)
dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}


#Defining Concerned states (starting states)

psi0=[0]

all_ones_index = 0
for ii in range (qubit_num):
    all_ones_index = all_ones_index + qubit_state_num**ii
#Defining states to include in the drawing of occupation
states_draw_list = [0,all_ones_index]
states_draw_names = ['00..0','11..1']

#Defining U (Target)

if is_dressed:
    
    gg = get_state_index(0,dressed_id) #[ee]
    ee = get_state_index(all_ones_index,dressed_id) #[eg]
    
    target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
    target_vec = target_vec +  (1)/np.sqrt(2)*v_c[:,gg] + (1)/np.sqrt(2)*v_c[:,ee]
    
    U = [target_vec]

else:   
    target_vec = np.zeros(qubit_state_num**qubit_num,dtype=complex)
    target_vec[0]=(1)/np.sqrt(2)
    target_vec[all_ones_index]=1/np.sqrt(2)

    U = [target_vec]



#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num**qubit_num)
U0= q_identity

#Defining control Hs

Hops = []
Hnames = []
ops_max_amp = []

Q_xx = multi_kron(Q_x,qubit_num)
Hops.extend([Q_xx])
Hnames.extend(['all x'])
ops_max_amp.extend([2.0])

print Hnames


#Defining convergence parameters
max_iterations = 100
decay = 500 #max_iterations/2
convergence = {'rate':0.10, 'update_step':10,'max_iterations':max_iterations,\
               'conv_target':1e-3,'learning_rate_decay':decay,'min_grad':1e-60}

reg_coeffs = {}



['all x']


In [3]:
#import h5py

#with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_ghz/00000_spin_chain_ghz_N8.h5','r') as hf:
#    u0 = np.array(hf.get('uks'))[31]

u0 = None

In [ ]:

uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names], 
                    state_transfer=True, use_gpu = False,
                    show_plots = False, Taylor_terms = [6,0],
       unitary_error = 1e-6, method = 'Adam',  maxA=ops_max_amp,\
                   dressed_info = dressed_info, reg_coeffs=reg_coeffs, file_name='nk_cpu2_spin_chain_ghz_N%d' %(qubit_num), initial_guess=u0,
                    data_path = '/home/nelson/Simulations/GRAPE-Data/benchmark')
      
      

Using 6 Taylor terms and 0 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.